In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# Classi base (da mantenere senza modifiche)
class Neuron:
    def __init__(self, num_inputs):
        self.weights = np.random.randn(num_inputs) * 0.01
        self.bias = np.random.randn() * 0.01
        
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.bias
        self.output = self.sigmoid(self.z)
        return self.output
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)

class Layer:
    def __init__(self, num_neurons, num_inputs):
        self.neurons = [Neuron(num_inputs) for _ in range(num_neurons)]
        
    def forward(self, inputs):
        self.outputs = []
        for neuron in self.neurons:
            output = neuron.forward(inputs)
            self.outputs.append(output)
        return np.array(self.outputs)

class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(1, len(layer_sizes)):
            layer = Layer(layer_sizes[i], layer_sizes[i-1])
            self.layers.append(layer)
    
    def forward(self, inputs):
        current_input = inputs
        for layer in self.layers:
            current_input = layer.forward(current_input)
        return current_input
    
    def backward(self, inputs, targets, learning_rate=0.1):
        # Forward pass per salvare tutti gli output
        layer_outputs = [inputs]
        current_input = inputs
        
        for layer in self.layers:
            current_input = layer.forward(current_input)
            layer_outputs.append(current_input.copy())
        
        # Backward pass
        # Calcola errore output layer
        output_errors = targets - layer_outputs[-1]
        layer_deltas = [output_errors * self.sigmoid_derivative(layer_outputs[-1])]
        
        # Propaga errore all'indietro
        for i in range(len(self.layers) - 2, -1, -1):
            layer_output = layer_outputs[i + 1]
            next_layer = self.layers[i + 1]
            
            # Calcola errore per layer corrente
            error = np.zeros(len(self.layers[i].neurons))
            for j, neuron in enumerate(self.layers[i].neurons):
                for k, next_neuron in enumerate(next_layer.neurons):
                    error[j] += layer_deltas[0][k] * next_neuron.weights[j]
            
            delta = error * self.sigmoid_derivative(layer_output)
            layer_deltas.insert(0, delta)
        
        # Aggiorna pesi e bias
        for i, layer in enumerate(self.layers):
            inputs_for_layer = layer_outputs[i]
            deltas_for_layer = layer_deltas[i]
            
            for j, neuron in enumerate(layer.neurons):
                # Aggiorna pesi
                for k in range(len(neuron.weights)):
                    neuron.weights[k] += learning_rate * deltas_for_layer[j] * inputs_for_layer[k]
                # Aggiorna bias
                neuron.bias += learning_rate * deltas_for_layer[j]
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def train(self, X, y, epochs=100, learning_rate=0.1, batch_size=32):
        losses = []
        accuracies = []
        
        for epoch in range(epochs):
            # Shuffle data
            indices = np.random.permutation(len(X))
            X_shuffled = X[indices]
            y_shuffled = y[indices]
            
            epoch_loss = 0
            # Mini-batch training
            for i in range(0, len(X), batch_size):
                batch_X = X_shuffled[i:i+batch_size]
                batch_y = y_shuffled[i:i+batch_size]
                
                batch_loss = 0
                for x, target in zip(batch_X, batch_y):
                    # Forward pass
                    output = self.forward(x)
                    
                    # Calcola loss (MSE)
                    loss = np.mean((target - output) ** 2)
                    batch_loss += loss
                    
                    # Backward pass
                    self.backward(x, target, learning_rate)
                
                epoch_loss += batch_loss / len(batch_X)
            
            # Calcola accuracy ogni 10 epoche
            if epoch % 10 == 0:
                accuracy = self.evaluate(X, y)
                losses.append(epoch_loss / (len(X) // batch_size))
                accuracies.append(accuracy)
                print(f"Epoch {epoch}: Loss = {losses[-1]:.4f}, Accuracy = {accuracy:.4f}")
        
        return losses, accuracies
    
    def evaluate(self, X, y):
        correct = 0
        for x, target in zip(X, y):
            output = self.forward(x)
            predicted = np.argmax(output)
            actual = np.argmax(target)
            if predicted == actual:
                correct += 1
        return correct / len(X)
    
    def predict(self, X):
        predictions = []
        for x in X:
            output = self.forward(x)
            predictions.append(np.argmax(output))
        return np.array(predictions)

# Carica e preprocessa MNIST
def load_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # Normalizza pixel values (0-1)
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    # Flatten immagini (28x28 -> 784)
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    
    # One-hot encoding per labels
    def to_one_hot(labels, num_classes=10):
        one_hot = np.zeros((len(labels), num_classes))
        for i, label in enumerate(labels):
            one_hot[i][label] = 1
        return one_hot
    
    y_train_one_hot = to_one_hot(y_train)
    y_test_one_hot = to_one_hot(y_test)
    
    return X_train, y_train_one_hot, X_test, y_test_one_hot, y_test

# Training e test
if __name__ == "__main__":
    print("Caricamento MNIST...")
    X_train, y_train, X_test, y_test_one_hot, y_test_labels = load_mnist()
    
    # Usa subset per training più veloce (prime 5000 immagini)
    X_train_subset = X_train[:5000]
    y_train_subset = y_train[:5000]
    
    # Usa subset per test (prime 1000 immagini)
    X_test_subset = X_test[:1000]
    y_test_subset = y_test_one_hot[:1000]
    y_test_labels_subset = y_test_labels[:1000]
    
    print("Creazione rete neurale...")
    # Architettura: 784 -> 300 -> 100 -> 10
    nn = NeuralNetwork([784, 300, 100, 10])
    
    print("Inizio training...")
    losses, accuracies = nn.train(
        X_train_subset, 
        y_train_subset,
        epochs=200,
        learning_rate=0.5,
        batch_size=32
    )
    
    print("\nValutazione finale...")
    train_accuracy = nn.evaluate(X_train_subset, y_train_subset)
    test_accuracy = nn.evaluate(X_test_subset, y_test_subset)
    
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Plot risultati
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(losses)
    plt.title('Training Loss')
    plt.xlabel('Epoch (x10)')
    plt.ylabel('Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(accuracies)
    plt.title('Training Accuracy')
    plt.xlabel('Epoch (x10)')
    plt.ylabel('Accuracy')
    
    plt.tight_layout()
    plt.show()
    
    # Test su alcune immagini
    print("\nTest su alcune immagini:")
    predictions = nn.predict(X_test_subset[:10])
    for i in range(10):
        print(f"Immagine {i}: Predetto = {predictions[i]}, Reale = {y_test_labels_subset[i]}")

2025-05-22 18:47:56.105054: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 18:47:56.695373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-22 18:47:56.698769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-22 18:47:58.632184: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Caricamento MNIST...
Creazione rete neurale...
Inizio training...
Epoch 0: Loss = 0.0913, Accuracy = 0.1100
